<a href="https://colab.research.google.com/github/dlwndms0812/Artificial_Intelligence/blob/main/homework3_2019110701.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.15

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./mnist/data/",one_hot="True")

In [3]:
#28*28픽셀=784개의 특징으로 이루어져있음
X=tf.placeholder(tf.float32,[None, 784])
#레이블은 0-9까지 10개
Y=tf.placeholder(tf.float32,[None,10])

keep_prob=tf.placeholder(tf.float32)

In [ ]:
#784(입력, 특징개수)->256(첫번째 은닉층)->256(두번째 은닉층)->256(세번째 은닉층)->10(결과값 0-9 분류 개수)
W1=tf.Variable(tf.random_normal([784,256], stddev=0.01))
L1=tf.nn.relu(tf.matmul(X,W1))
L1=tf.nn.dropout(L1, keep_prob) 

W2=tf.Variable(tf.random_normal([256,256], stddev=0.01))
L2=tf.nn.relu(tf.matmul(L1,W2))
L2=tf.nn.dropout(L2,keep_prob)

W3=tf.Variable(tf.random_normal([256,256], stddev=0.01))
L3=tf.nn.relu(tf.matmul(L2,W3))
L3=tf.nn.dropout(L3,keep_prob)

W4=tf.Variable(tf.random_normal([256,10],stddev=0.01))
model=tf.matmul(L3,W4)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                       logits=model,labels=Y))
#GradientDescentOptimizer 사용 수치 0.1로 조정
optimizer=tf.train.GradientDescentOptimizer(0.1).minimize(cost)

In [ ]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

for epoch in range(30):
  total_cost=0

  for i in range(total_batch):
    batch_xs, batch_ys=mnist.train.next_batch(batch_size)
    _, cost_val=sess.run([optimizer,cost],
                         feed_dict={X:batch_xs,
                                    Y:batch_ys,
                                    keep_prob:0.8}) #드롭아웃 수치 0.8
    total_cost+=cost_val
  
  print('Epoch:', '%04d' %(epoch+1),
        'Avg. cost=', '{:.3f}'.format(total_cost/total_batch))
  
print('최적화 완료!')

In [ ]:
is_correct=tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:',sess.run(accuracy,
                      feed_dict={X:mnist.test.images,
                                 Y:mnist.test.labels,
                                 keep_prob:1}))